In [8]:
pip install transformers torch accelerate tqdm


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install --upgrade bitsandbytes

Defaulting to user installation because normal site-packages is not writeable
  Using cached bitsandbytes-0.45.0-py3-none-manylinux_2_24_x86_64.whl.metadata (2.9 kB)
Using cached bitsandbytes-0.45.0-py3-none-manylinux_2_24_x86_64.whl (69.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade transformers

Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 83.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 94.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade accelerate

Defaulting to user installation because normal site-packages is not writeable
  Using cached accelerate-1.2.0-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.2.0-py3-none-any.whl (336 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install --upgrade datasets

Defaulting to user installation because normal site-packages is not writeable
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.1 MB/s eta 0:00:00
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.8 MB/s eta 0:00:00
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.

In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from datasets import load_dataset
import re
from tqdm import tqdm
import os

In [4]:
print("CUDA Available:", torch.cuda.is_available())
print("Current Device:", torch.cuda.current_device())
print("Device Name:", torch.cuda.get_device_name(torch.cuda.current_device()))


CUDA Available: True
Current Device: 0
Device Name: NVIDIA A100-SXM4-40GB


In [2]:
from huggingface_hub import login

# Replace 'your_token_here' with the token you copied
login("XXXXXXXXXXXXXXXXXXXXXXX")


In [5]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from datasets import load_dataset
import re
from tqdm import tqdm
import os


# Load the LLaMA-3.1-8B-Instruct model and tokenizer
model_path = "nkulka29/Llama-3.1-8B"  # Fine Tuned Model

# Load model with quantization configuration and multi-GPU support
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",
    device_map="auto",
    quantization_config=quantization_config
)

# Compile the model for optimized performance (PyTorch 2.0+)
model = torch.compile(model)

# Load the harmless dataset
hh = load_dataset("Anthropic/hh-rlhf")

# Convert the train and test datasets to DataFrames and combine them
hh_train_df = hh['train'].to_pandas()
hh_test_df = hh['test'].to_pandas()
hh_combined = pd.concat([hh_train_df, hh_test_df], axis=0)

# Function to extract the first Human prompt
def extract_first_prompt(text):
    prompt_pattern = r"Human: (.*?)(\n\n|$)"
    match = re.search(prompt_pattern, text, re.DOTALL)
    return match.group(1).strip() if match else None

# Create the new DataFrame with the extracted prompts
hh_df = pd.DataFrame({
    'prompt': hh_combined['chosen'].apply(extract_first_prompt)
})

# Sample 10,000 rows and reset the index
df = hh_df.sample(n=4000, random_state=18).reset_index(drop=True)

# System prompt to ensure safe and helpful responses
system_prompt = (
    "You are an AI assistant. Your goal is to provide responses that are accurate, safe, and helpful. "
    "Ensure that your answers are informative and respectful."
)

# Function to generate responses directly with the compiled model
def generate_responses(prompts, batch_size=32, temperature=0.7, max_new_tokens=100):
    responses = []
    for i in tqdm(range(0, len(prompts), batch_size), desc="Generating Batches"):
        batch = prompts[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(model.device)
        
        # Generate responses with the compiled model
        with torch.no_grad():
            output_sequences = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id
            )
        
        # Decode and append responses
        batch_responses = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)
        responses.extend([response.split("[ASSISTANT]:")[-1].strip() for response in batch_responses])
    
    return responses


for i in range(1, 6):  # Generate 5 responses for each prompt
    print(f"Generating Response {i}...")
    full_prompts = [
        f"[SYSTEM]: {system_prompt}\n[USER]: {prompt}\n[ASSISTANT]:"
        for prompt in df["prompt"]
    ]
    
    responses = generate_responses(full_prompts)
    df[f"response_{i}"] = responses

    # Save progress after every 100 fully completed rows
    if i % save_interval == 0 or i == 5:
        df.to_csv(f"generated_responses_checkpoint.csv", index=False)
        print(f"Saved progress to 'generated_responses_checkpoint.csv' after completing response_{i}")



tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

/packages/envs/pytorch-gpu-2.1.0-cuda-12.1/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/packages/envs/pytorch-gpu-2.1.0-cuda-12.1/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


...

Saved progress to 'generated_responses_checkpoint.csv' after completing response_3
Generating Response 4...


Generating Batches: 100%|██████████| 32/32 [03:16<00:00,  6.15s/it]


Saved progress to 'generated_responses_checkpoint.csv' after completing response_4
Generating Response 5...


Generating Batches: 100%|██████████| 32/32 [03:16<00:00,  6.16s/it]

Saved progress to 'generated_responses_checkpoint.csv' after completing response_5
